#  Introduction to Operations Research -  Fredrick S. Hillier, Gerald J Lieberman

In this notebook I will demonstrate how to use the [Julia Programming Language](https://julialang.org) and the powerful [JuMP](https://jump.dev) optimization framework to solve common business problems using Operations Research techniques.

This notebook assumes you are somewhat familiar with basic Julia syntax and things like functions, macros, packages, etc.
<!--Much of the recent hype surrounding Machine learning has drawn attention away from these well established and highly valuable problem solving techniques. I was originally drawn to study operations research while working as a Data Scientist for a large oil company.  Within the organization there was a huge push for data science to revifify a struggling business (now in chapter 11 bankruptcy...) and managers were eager to demonstrate machine learning had a role to play in the business' resurrection.  My job was to apply data science techniques throughout the entire organization, wherever valuable opportunities might present themselves.-->


<!---Shamefully absent from the many "How to be a Data Scientist" medium posts is quality advice on how to successfully add value within your business. Speaking from experience it is incredibly challenging, particularly when starting from scratch, *but*  there is an entire 70+ year old field that had been studying how to do it... For some reason, NOBODY TALKS ABOUT IT!  Ask yourselves, in all your data science queries on the internet, twitter feeds, etc how often do you hear people say "Operations Research" or (synonym) "Management Science?"  I'd be shocked if anyone responded "often," yet that is precisely what the field is about.  Not only does it study how to perform scientific research on business processes, but how to grapple with very large and complex problems within the business.  I'm talking things like huge infrastructure projects, manufacturing projects, aerospace and defense projects. The types of projects where success can have huge impact on the bottom line. Naturally the size of these projects drew investment into research so as to increase the probability of success.

So there I was, thinking deeply about how I was going to be successful in my new role when I stumbled upon a huge body of literature about how to do exactly what I wanted.  The amount of research is tremendous, so I would suggest against diving in straight away. Instead, start with a textbook like the one above that will more gently carry you through the significant advancements, and dive more deeply only when necessary for a specific problem.  Introduction to Operations Research is FULL of case studies and links to even more case studies where businesses saved or made $10_000_000+ with the very same techniques I will demonstrate. 

With that in mind, lets get started learning how to apply the great Julia language to solving these problems in the workplace. I'll start with some simple examples of "linear programming" which is the class of problems that includes resource allocation, and we'll move on to more challenging problems from there. --->

If you don't have the packages installed in the next cell, uncomment and run.

In [3]:
# Using Pkg
# Pkg.add("Clp")
# Pkg.add("Cbc")
# Pkg.add("GLPK")

m = Model(GLPK.Optimizer)

In [4]:
# Lets begin by importing the necessary packages
using JuMP
using GLPK #GLPK is an open source optimizing software

## Problem #1 Set up

>The WYNDOR GLASS CO. produces high quality glass products, including windows and glass doors. It has three plants. Aluminum frames and hardware are made in Plant 1, wood frames are made in Plant 2, and Plant 3 produces the glass and assembles the products.
>
>Because of declining earnings, top management has decided to revamp the company's product line. Unprofitable products are being discontinued, releasing production capacity to launch two new products having large sales potential:
>* Product 1: An 8-foot glass door with aluminum framing
>* Product 2: A 4x6 foot double-hung wood-framed window
>
>Product 1 requires some of the production capacity in Plants 1 and 3, but none in Plant 2. Product 2 needs only Plants 2 and 3. The marketing division has concluded that the company could sell as much of either product as could be produced by these plants. However, because both products would be competing for the same production capacity in Plant 3, its not clear which mix of the two products would be *most profitable*.
>
>**Determine what the production rates should be for the two products in order to maximize their total profit, subject to the restrictions imbosed by the limited production capacities available in the three plants. (each product will be produced in batches of 20, so the production rate is defined as the number of batches produced per week.) Any combination of production rates that satisfies these restrictions is permitted, including producing none of one product and as much as possible of the other.**


> The Operations Research team has identified the data that need to be gathered:
1. Number of hours of production time available per week in each plant for the products.
1. Number of hours of production time used in each plant for each batch produced of the new product
1. Profit per batch produced of each new product.

## Data

The Operations Research Team discovers that Plant 1 has only 4 hours of available production time per week, Plant 2 has 12, and Plant 3 has 18. Additionally, they learn that a batch of 20 units (the minimum batch size) takes 1 hour at Plant 1 and 3 hours at Plant 3. Product 2 takes 2 hours at Plant 2 and 2 hours at Plant 3.  Finally they learn that the Profit per batch of product is 3,000 for Product 1 and 5,000 for product 2. The data is summarized in the table below.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-0pky">Plant</th>
    <th class="tg-0pky">Product 1 Production Time</th>
    <th class="tg-0pky">Product 2 Production time</th>
    <th class="tg-0pky">Availabel time per week</th>
    <th class="tg-0pky"></th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0pky">1</td>
    <td class="tg-0pky">1</td>
    <td class="tg-0pky">0</td>
    <td class="tg-0pky">4</td>
    <td class="tg-0pky"></td>
  </tr>
  <tr>
    <td class="tg-0pky">2</td>
    <td class="tg-0pky">0</td>
    <td class="tg-0pky">2</td>
    <td class="tg-0pky">12</td>
    <td class="tg-0pky"></td>
  </tr>
  <tr>
    <td class="tg-0pky">3</td>
    <td class="tg-0pky">3</td>
    <td class="tg-0pky">2</td>
    <td class="tg-0pky">18</td>
    <td class="tg-0pky"></td>
  </tr>
  <tr>
    <td class="tg-0pky">Profit</td>
    <td class="tg-0pky">3000</td>
    <td class="tg-0pky">5000</td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
  </tr>
</tbody>
</table>

m = Model(with_optimizer(GLPK.Optimizer))

In [42]:
# Define the variables used in our model.
# x₁ is the number of batches of product 1
# x₂ is the number of batches of product 2
# We only want to deal with full batches
# so we set the type to Int
@variable(m, x₁ ≥ 0, Int)
@variable(m, x₂ ≥ 0, Int)

# Now we define our objective. We wish to maximize
# our profit Z = 3x₁ + 5x₂ in thousands of dollars
@objective(m, Max, 3x₁ + 5x₂)

# Finally, lets specify our constraints. Production capacity.
@constraint(m, x₁ ≤ 4)
@constraint(m, 2x₂ ≤ 12)
@constraint(m, 3x₁ + 2x₂ ≤ 18)

3 x₁ + 2 x₂ ≤ 18.0

In [43]:
# Now just call optimize! which modifies our model, m, in place
optimize!(m)

In [52]:
println(m)
println("x₁ = ", value(x₁))
println("x₂ = ", value(x₂))

Max 3 x₁ + 5 x₂
Subject to
 x₁ ≤ 4.0
 2 x₂ ≤ 12.0
 3 x₁ + 2 x₂ ≤ 18.0
 x₁ ≥ 0.0
 x₂ ≥ 0.0
 x₁ integer
 x₂ integer

x₁ = 2.0
x₂ = 6.0


What we have just solved is called a Linear Integer Program. *Linear* because our objective and constraints are linear functions of our variables $xᵢ$ (conventionally referred to as *decision variables*. *Integer* because we only want solutions that are integer values of our variables.  Typically integer programs are significantly more difficult to solve than standard linear programs, but with the size of this model and Julia's performance, it's nothing to sweat about.

Integer programs are common in manufacturing where we gain no value in producing half a car, or launching 1/3 of a satellite into space.

Also notice that we have a nonnegativity constraint on our decision variables as it is not meaningful to discuss making negative quantities of product.